In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import graphviz 
# might need to install -- need *both* python and system
# so use Anaconda (or pip) to install graphviz, and also https://graphviz.org/download/
# this was a non-trivial install on Mac (depends on Xcode command-line tools)
# `pip install graphviz` and `brew install graphviz`
import datetime as dt

%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import confusion_matrix, mean_squared_error, mean_absolute_error

In [13]:
bb = pd.read_csv("../../Data/1999_2021/aggragate_daily_data_2000to2020.csv", index_col = "date")[['influent_flow']].loc['2008-01-01':'2017-12-31']

weather = pd.read_csv("../../Data/historic_weather_all_SP_daily.csv", index_col = "date")



In [4]:
'''#really ugly. sorry.
weather_0 = weather_tmp.rename(columns={'rain_mm':'rain_mm_0', 'temp_max':'temp_max_0', 'temp_min':'temp_min_0'}).swaplevel(axis=1).sort_index(axis=1)
weather_7 = weather_tmp.shift(7).rename(columns={'rain_mm':'rain_mm_7', 'temp_max':'temp_max_7', 'temp_min':'temp_min_7'}).swaplevel(axis=1).sort_index(axis=1)
weather_14 = weather_tmp.shift(14).rename(columns={'rain_mm':'rain_mm_14', 'temp_max':'temp_max_14', 'temp_min':'temp_min_14'}).swaplevel(axis=1).sort_index(axis=1)
weather_30 = weather_tmp.shift(30).rename(columns={'rain_mm':'rain_mm_30', 'temp_max':'temp_max_30', 'temp_min':'temp_min_30'}).swaplevel(axis=1).sort_index(axis=1)
'''

"#really ugly. sorry.\nweather_0 = weather_tmp.rename(columns={'rain_mm':'rain_mm_0', 'temp_max':'temp_max_0', 'temp_min':'temp_min_0'}).swaplevel(axis=1).sort_index(axis=1)\nweather_7 = weather_tmp.shift(-7).rename(columns={'rain_mm':'rain_mm_7', 'temp_max':'temp_max_7', 'temp_min':'temp_min_7'}).swaplevel(axis=1).sort_index(axis=1)\nweather_14 = weather_tmp.shift(-14).rename(columns={'rain_mm':'rain_mm_14', 'temp_max':'temp_max_14', 'temp_min':'temp_min_14'}).swaplevel(axis=1).sort_index(axis=1)\nweather_30 = weather_tmp.shift(-30).rename(columns={'rain_mm':'rain_mm_30', 'temp_max':'temp_max_30', 'temp_min':'temp_min_30'}).swaplevel(axis=1).sort_index(axis=1)\n"

## Change .fillna(0) to something more sophisticated?

In [33]:
# rain = weather.pivot(columns='station', values=['rain_mm']).fillna(0)[[('rain_mm',    'A701 SAO PAULO - MIRANTE'), ('rain_mm',               'A713 SOROCABA'), ('rain_mm',      'A744 BRAGANCA PAULISTA'),  ('rain_mm',                'A755 BARUERI'), ('rain_mm',             'A711 SAO CARLOS'), ('rain_mm',           'A741 BARRA BONITA')]]
rain = weather.pivot(columns='station', values=['rain_mm']).fillna(0)[[('rain_mm',    'A701 SAO PAULO - MIRANTE'), ('rain_mm',               'A713 SOROCABA'), ('rain_mm',             'A711 SAO CARLOS'), ('rain_mm',           'A741 BARRA BONITA')
]]

In [34]:
rain_agg = bb

for i in range(0,10):
    rain_lag = rain.shift(i).rename(columns={'rain_mm':'rain_mm_' + str(i)})
    rain_agg = rain_agg.join(rain_lag).dropna()

/Users/jzhu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
/Users/jzhu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
/Users/jzhu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
/Users/jzhu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)
/Users/jzhu/opt/anaconda3/lib/python3.8/site-packages/pa

In [36]:
X_agg = rain_agg.drop('influent_flow', axis = 1)
y_agg = rain_agg.influent_flow
X_agg_train, X_agg_test, y_agg_train, y_agg_test = train_test_split(X_agg, y_agg, train_size = 0.8, random_state = 0)

## This is all stations with lags of 1 through 7. 

In [37]:
# All stations, all lags predicting energy generated at B. Bonita

random_forest_rain_agg = RandomForestRegressor(max_features = 3, random_state = 17)

random_forest_rain_agg.fit(X_agg_train, y_agg_train)

pred_agg = random_forest_rain_agg.predict(X_agg_test)

Importance_agg = pd.DataFrame({'Importance':random_forest_rain_agg.feature_importances_*100}, 
                          index = X_agg.columns)

Importance_agg = Importance_agg.sort_values(by = 'Importance', axis = 0)

print("random forest MAE:", mean_absolute_error(y_agg_test, pred_agg))

random forest MAE: 138.11137950905155


In [38]:
Importance_agg = pd.DataFrame({'Importance':random_forest_rain_agg.feature_importances_*100}, 
                          index = X_agg.columns)

Importance_agg = Importance_agg.sort_values(by = 'Importance', axis = 0)

In [41]:
Importance_agg.sort_values('Importance',ascending=False).head(20)

,Importance
"(rain_mm_2, A701 SAO PAULO - MIRANTE)",5.632505
"(rain_mm_3, A711 SAO CARLOS)",5.301806
"(rain_mm_2, A713 SOROCABA)",5.232183
"(rain_mm_3, A701 SAO PAULO - MIRANTE)",4.591902
"(rain_mm_2, A711 SAO CARLOS)",4.118011
"(rain_mm_4, A711 SAO CARLOS)",3.480948
"(rain_mm_4, A701 SAO PAULO - MIRANTE)",3.475303
"(rain_mm_1, A711 SAO CARLOS)",3.427332
"(rain_mm_3, A713 SOROCABA)",3.386451
"(rain_mm_4, A713 SOROCABA)",3.135665


In [40]:
Importance_agg.sort_values('Importance',ascending=False).tail(20)

,Importance
"(rain_mm_6, A713 SOROCABA)",2.101536
"(rain_mm_5, A741 BARRA BONITA)",1.998934
"(rain_mm_6, A741 BARRA BONITA)",1.884088
"(rain_mm_0, A711 SAO CARLOS)",1.840414
"(rain_mm_6, A711 SAO CARLOS)",1.744968
"(rain_mm_0, A741 BARRA BONITA)",1.687426
"(rain_mm_7, A701 SAO PAULO - MIRANTE)",1.636347
"(rain_mm_0, A701 SAO PAULO - MIRANTE)",1.584139
"(rain_mm_9, A701 SAO PAULO - MIRANTE)",1.485969
"(rain_mm_7, A713 SOROCABA)",1.466480


## Geographically relevant stations but only lagged 7 days

KeyError: 'influent_flow'

In [ ]:
Importance_7.sort_values('Importance', ascending=False).head(10)

In [ ]:
Importance_7.sort_values('Importance', ascending=False).tail(10)

## All stations lagged 14 days

In [ ]:
#Rain lagged 14 days predicting energy generated at B. Bonita

X_14 = rain_14.drop('influent_flow', axis = 1)
y_14 = rain_14.influent_flow
X_14_train, X_14_test, y_14_train, y_14_test = train_test_split(X_14, y_14, train_size = 0.8, random_state = 0)

random_forest_rain_14 = RandomForestRegressor(max_features = 10, random_state = 1)

random_forest_rain_14.fit(X_14_train, y_14_train)

pred_14 = random_forest_rain_14.predict(X_14_test)

Importance_14 = pd.DataFrame({'Importance':random_forest_rain_14.feature_importances_*100}, 
                          index = X_14.columns)

Importance_14 = Importance_14.sort_values(by = 'Importance', axis = 0)

print("random forest MAE:", mean_absolute_error(y_14_test, pred_14))


In [ ]:
Importance_14.sort_values('Importance',ascending=False).head(10)

In [ ]:
Importance_14.sort_values('Importance',ascending=False).tail(10)

## All stations lagged 30 days

In [ ]:
X_30 = rain_30.drop('influent_flow', axis = 1)
y_30 = rain_30.influent_flow
X_30_train, X_30_test, y_30_train, y_30_test = train_test_split(X_30, y_30, train_size = 0.8, random_state = 0)

random_forest_rain_30 = RandomForestRegressor(max_features = 10, random_state = 1)

random_forest_rain_30.fit(X_30_train, y_30_train)

pred_30 = random_forest_rain_30.predict(X_30_test)

Importance_30 = pd.DataFrame({'Importance':random_forest_rain_30.feature_importances_*100}, 
                          index = X_30.columns)

Importance_30 = Importance_30.sort_values(by = 'Importance', axis = 0)

print("random forest MAE:", mean_absolute_error(y_30_test, pred_30))


In [ ]:
Importance_30.sort_values('Importance',ascending=False).head(10)


In [ ]:
Importance_30.sort_values('Importance',ascending=False).tail(10)